In [65]:
# initializing

import pandas as pd
import pandas_gbq as pgbq
from multiprocessing import Pool
from scipy import stats
import os
import time

directory = '/Users/joshharris/community_health/sql/community_facts'
sql_modules = list(map(lambda x: os.path.splitext(x)[0], os.listdir(directory)))
google_project_id = 'solid-ridge-104914'
dataset = 'community_networks'

In [76]:
#if community facts are already saved, use those otherwise re-calculate and save

def bigquery_worker(i):
    sql_module_type = os.path.basename(directory)
    sql_module = sql_modules[i]
    print('Starting module ',i,': ',sql_module, sep='')
    try:
        result = pgbq.read_gbq('select * from '+dataset+'.'+sql_module_type+'_'+sql_module, google_project_id, dialect='standard', verbose=False)
        print(sql_module_type,'_',sql_module,' table found.', sep='')
    except:
        print(+sql_module_type+'_'+sql_module+' table does not exist, generating now.')
        with open(directory+'/'+sql_module+'.txt') as query_file:
            query = query_file.read()
        result = pgbq.read_gbq(query, google_project_id, dialect='standard', verbose=False)
        print('Saving '+dataset+'.'+sql_module_type+'_'+sql_module)
        pgbq.to_gbq(result,  dataset+'.'+sql_module_type+'_'+sql_module, google_project_id, if_exists='replace', verbose=True)
    return (result) 
        

In [77]:
#Run community facts sql modules and join them together

if __name__ == '__main__':
    with Pool(processes=len(sql_modules)) as pool:
        result = pool.map(bigquery_worker, (range(len(sql_modules),)))
        pool.close()
        pool.join()
        
joined = result[0].set_index('cid', drop=False)
print('Joining modules', sep='')
for i in range(len(result)-1):
    joined = joined.join(result[i+1].set_index('cid'), how='left', lsuffix='_left')
print('Saving '+dataset+'.'+sql_module_type+'_main')
pgbq.to_gbq(joined, dataset+'.'+sql_module_type+'_main', google_project_id, if_exists='replace', verbose=False )
print('Done', sep='')

Starting module 1: m1_total_retention
Starting module 3: mau_june
Starting module 4: names_and_nodes
Starting module 8: w4_paid_retention
Starting module 0: community_age
Starting module 6: paid_signups_june
Starting module 7: signups_june
Starting module 5: new_listing_ntiles
Starting module 9: w4_total_retention
Starting module 2: m3_total_retention


KeyboardInterrupt: 

In [75]:
pgbq.to_gbq(joined, dataset+'.'+sql_module_type+'_main', google_project_id, if_exists='replace', verbose=False )